In [ ]:
import numpy as np
import plotly.graph_objects as go
import itertools
import pandas as pd
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
# Function to generate a cylinder with variable radius

def generate_slipstream(rad, z_sections, radius_funcs, resolution_theta=50, resolution_z=50):
    z_min, z_max = min(z_sections), max(z_sections)
    z_steps = np.linspace(z_min, z_max, resolution_z)
    theta = np.linspace(0, 2 * np.pi, resolution_theta)

    radius = np.zeros_like(z_steps)
    for i in range(len(z_sections) - 1):
        mask = (z_steps >= z_sections[i]) & (z_steps <= z_sections[i + 1])
        radius[mask] = rad * radius_funcs[i](z_steps[mask])

    # Avoid creating full grids
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    x = radius[:, None] * cos_theta
    y = radius[:, None] * sin_theta
    z = (z_steps * rad * 2)[:, None] + np.zeros_like(theta)

    # Combine points efficiently
    points = np.column_stack((x.ravel(), y.ravel(), z.ravel()))
    return points


# Define radius functions
#def radius_above(z): return 1 + z *0.25
#def radius_contract(z): return 1 + 0*z
#def radius_nf(z): return 1 - (z+0.9)*0.4
#def radius_ff(z): return 1.3 - (z+1.6)*0.3
#radius_funcs = [radius_ff, radius_nf, radius_contract, radius_above]

def radius_above(z): return 1 +z*0
def radius_contract(z): return 1 + z*0
radius_funcs = [radius_above, radius_contract]

# Function to generate surface grids for plotting
def generate_surface_grids(points, resolution_theta, resolution_z):
    x, y, z = points[:, 0], points[:, 1], points[:, 2]
    x_grid = x.reshape(resolution_z, resolution_theta)
    y_grid = y.reshape(resolution_z, resolution_theta)
    z_grid = z.reshape(resolution_z, resolution_theta)
    return x_grid, y_grid, z_grid


# Function to create rotation matrices for given angles
def create_transformation_matrix(a1, a2, a3):
    TI1_1 = np.array([
        [1, 0, 0, 0],
        [0, np.cos(a1), -np.sin(a1), 0],
        [0, np.sin(a1), np.cos(a1), 0],
        [0, 0, 0, 1]
    ])
    T1_2 = np.array([
        [1, 0, 0, armlength],
        [0, 1, 0, 0],
        [0, 0, 1, z_offset],
        [0, 0, 0, 1]
    ])

    TI2_1 = np.array([
        [np.cos(np.pi / 3), -np.sin(np.pi / 3), 0, 0],
        [np.sin(np.pi / 3), np.cos(np.pi / 3), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    T2_2 = np.array([
        [1, 0, 0, armlength],
        [0, np.cos(a2), -np.sin(a2), 0],
        [0, np.sin(a2), np.cos(a2), 0],
        [0, 0, 0, 1]
    ])
    TI2_3 = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, z_offset],
        [0, 0, 0, 1]
    ])

    TI3_1 = np.array([
        [np.cos(-np.pi / 3), -np.sin(-np.pi / 3), 0, 0],
        [np.sin(-np.pi / 3), np.cos(-np.pi / 3), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    T3_2 = np.array([
        [1, 0, 0, armlength],
        [0, np.cos(a3), -np.sin(a3), 0],
        [0, np.sin(a3), np.cos(a3), 0],
        [0, 0, 0, 1]
    ])
    TI3_3 = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, z_offset],
        [0, 0, 0, 1]
    ])


    TI1 = np.matmul(TI1_1,T1_2)
    TI2 = np.matmul(np.matmul(TI2_1, T2_2), TI2_3)
    TI3 = np.matmul(np.matmul(TI3_1, T3_2), TI3_3)
    return TI1, TI2, TI3

def invert_transformation(matrix):
    """
    Computes the inverse of a 4x4 homogeneous transformation matrix.
    
    Args:
        T (np.ndarray): A 4x4 homogeneous transformation matrix.
    
    Returns:
        np.ndarray: The inverse of the input transformation matrix.
    """
    # Extract rotation matrix (3x3) and translation vector (3x1)
    R = matrix[:3, :3]
    t = matrix[:3, 3]
    
    # Inverse of rotation is just the transpose of R
    R_inv = R.T
    
    # Inverse of translation is -R_inv * t
    t_inv = -np.dot(R_inv, t)
    
    # Construct the inverse transformation matrix
    matrix_inv = np.eye(4)
    matrix_inv[:3, :3] = R_inv
    matrix_inv[:3, 3] = t_inv
    
    return matrix_inv


# Transformation matrix
def apply_transform(matrix, points):
    num_points = points.shape[0]
    homogeneous_points = np.c_[points, np.ones(num_points)]  # Inline addition of the homogeneous coordinate
    return np.dot(homogeneous_points, matrix.T)[:, :3]


def find_intersection_points(cylinder1_kdtree, cylinder1_points, cylinder2_points, ti1, ti2, t1i, t2i, threshold=0.1):
    """
    Finds intersection points between two cylinders, transforms them back, and returns the closest Z points.
    
    Args:
        cylinder1_kdtree: KDTree for Cylinder 1.
        cylinder1_points: Points of Cylinder 1 in its local frame.
        cylinder2_points: Points of Cylinder 2 in its local frame.
        ti1_inv: Inverse transformation matrix for Cylinder 1.
        ti2_inv: Inverse transformation matrix for Cylinder 2.
        threshold: Distance threshold to consider points intersecting.
    
    Returns:
        - Transformed intersection points back into the inertial frame.
        - Closest point with minimum +Z and -Z for both cylinders.
        - Z values and coordinates in the inertial frame.
    """

    # Step 1: Query KDTree for intersections within a threshold
    distances, indices = cylinder1_kdtree.query(cylinder2_points, distance_upper_bound=threshold)
    valid_mask = distances < threshold
    intersection_points_cyl2 = cylinder2_points[valid_mask]

    # Step 2: Transform intersection points back to Cylinder 1 and 2 local coordinates
    intersection_points_cyl1 = cylinder1_points[indices[valid_mask]]
    if len(intersection_points_cyl1) == 0:
        return [], None, None, None, None, None, None, None, None

    # Transform using inverse matrices to local cylinder coordinates
    intersection_points_cyl1_transformed = apply_transform(t1i,intersection_points_cyl1)
    intersection_points_cyl2_transformed = apply_transform(t2i,intersection_points_cyl2)
    # Step 3: Find min +Z and -Z values and corresponding points
    #print(intersection_points_cyl1)
    #print(intersection_points_cyl1_transformed)

    # Initialize the result variables in case no valid points are found
    min_cyl1 = max_cyl1 = None
    min_cyl1_idx = max_cyl1_idx = None
    min_cyl2 = max_cyl2 = None
    min_cyl2_idx = max_cyl2_idx = None
    min_distance_cyl_1 = max_distance_cyl_1 = min_distance_cyl_2 = max_distance_cyl_2 = None


    if np.any(intersection_points_cyl1_transformed):
        min_cyl1_idx = np.argmin(intersection_points_cyl1_transformed[:, 2])
        min_cyl1 = intersection_points_cyl1_transformed[min_cyl1_idx].reshape(1, 3)
        min_distance_cyl_1 = min_cyl1[0, 2]

        max_cyl1_idx = np.argmax(intersection_points_cyl1_transformed[:, 2])
        max_cyl1 = intersection_points_cyl1_transformed[max_cyl1_idx].reshape(1, 3)
        max_distance_cyl_1 = max_cyl1[0, 2]


    if np.any(intersection_points_cyl2_transformed):
        min_cyl2_idx = np.argmin(intersection_points_cyl2_transformed[:, 2])
        min_cyl2 = intersection_points_cyl2_transformed[min_cyl2_idx].reshape(1, 3)
        min_distance_cyl_2 = min_cyl2[0, 2]

        max_cyl2_idx = np.argmax(intersection_points_cyl2_transformed[:, 2])
        max_cyl2 = intersection_points_cyl2_transformed[max_cyl2_idx].reshape(1, 3)
        max_distance_cyl_2 = max_cyl2[0, 2]




    # Step 3: Transform the points back to the inertial frame, if they were found
    min_cyl1_inertial = apply_transform(ti1, min_cyl1) if min_cyl1 is not None else None
    #print(min_negative_z_cyl1.shape)
    #print(min_negative_z_cyl1)
    max_cyl1_inertial = apply_transform(ti1, max_cyl1) if max_cyl1 is not None else None
    min_cyl2_inertial = apply_transform(ti2, min_cyl2) if min_cyl2 is not None else None
    max_cyl2_inertial = apply_transform(ti2, max_cyl2) if max_cyl2 is not None else None

    return (
        intersection_points_cyl1,
        min_distance_cyl_1,
        max_distance_cyl_1,
        min_distance_cyl_2,
        max_distance_cyl_2,
        min_cyl1_inertial,
        max_cyl1_inertial,
        min_cyl2_inertial,
        max_cyl2_inertial
    )

def downsample_grid(grid, factor):
    return grid[::factor, ::factor]


# Function to plot the cylinders, intersection points, and closest points
def plot_cylinders(cylinder1_surface, cylinder2_surface, cylinder3_surface,
                    intersection_points_cyl1_2, intersection_points_cyl1_3,
                    min_distance_cyl_1_2, max_distance_cyl_1_2,
                    min_distance_cyl_2_1, max_distance_cyl_2_1,
                    min_distance_cyl_1_3, max_distance_cyl_1_3,
                    min_distance_cyl_3_1, max_distance_cyl_3_1, 
                    min_cyl1_2_inertial, max_cyl1_2_inertial, 
                    min_cyl2_1_inertial, max_cyl2_1_inertial,
                    min_cyl1_3_inertial, max_cyl1_3_inertial, 
                    min_cyl3_1_inertial, max_cyl3_1_inertial,
                    center1, center2, center3, a1, a2, a3):
    fig = go.Figure()

    # Add Cylinder 1 as a surface
    fig.add_trace(go.Surface(
        x=cylinder1_surface[0], y=cylinder1_surface[1], z=cylinder1_surface[2],
        colorscale='Greens',
        opacity=0.7,
        showscale=False,
        name=f'Cylinder 1'
    ))

    # Add Cylinder 2 as a surface
    fig.add_trace(go.Surface(
        x=cylinder2_surface[0], y=cylinder2_surface[1], z=cylinder2_surface[2],
        colorscale='Reds',
        opacity=0.7,
        showscale=False,
        name=f'Cylinder 2'
    ))

    # Add Cylinder 3 as a surface
    fig.add_trace(go.Surface(
        x=cylinder3_surface[0], y=cylinder3_surface[1], z=cylinder3_surface[2],
        colorscale='Blues',
        opacity=0.7,
        showscale=False,
        name=f'Cylinder 3'
    ))

    # Add vector from origin to Cylinder 1 center
    fig.add_trace(go.Scatter3d(
        x=[0, 271.5 + 124], y=[0, 0], z=[0, 0],
        mode='lines+markers',
        marker=dict(size=5, color='green'),
        line=dict(color='green', width=5),
        name='Vector to Arm 1'
    ))
    fig.add_trace(go.Scatter3d(
        x=[271.5 + 124, center1[0,0]], y=[0, center1[0,1]], z=[0, center1[0,2]],
        mode='lines+markers',
        marker=dict(size=5, color='green'),
        line=dict(color='green', width=5),
        name='Vector to Arm 1'
    ))

    # Add vector from origin to Cylinder 2 center
    fig.add_trace(go.Scatter3d(
        x=[0, (271.5 + 124) * np.cos(np.pi / 3)], y=[0, (271.5 + 124) * np.sin(np.pi / 3)], z=[0, 0],
        mode='lines+markers',
        marker=dict(size=5, color='red'),
        line=dict(color='red', width=5),
        name='Vector to Arm 2'
    ))
    fig.add_trace(go.Scatter3d(
        x=[(271.5 + 124) * np.cos(np.pi / 3), center2[0,0]], y=[(271.5 + 124) * np.sin(np.pi / 3), center2[0,1] ], z=[0, center2[0,2]],
        mode='lines+markers',
        marker=dict(size=5, color='red'),
        line=dict(color='red', width=5),
        name='Vector to Arm 2'
    ))

    # Add vector from origin to Cylinder 3 center
    fig.add_trace(go.Scatter3d(
        x=[0, (271.5 + 124) * np.cos(5*np.pi / 3)], y=[0, (271.5 + 124) * np.sin(5*np.pi / 3)], z=[0, 0],
        mode='lines+markers',
        marker=dict(size=5, color='blue'),
        line=dict(color='blue', width=5),
        name='Vector to Arm 3'
    ))
    fig.add_trace(go.Scatter3d(
        x=[(271.5 + 124) * np.cos(5*np.pi / 3), center3[0,0]], y=[(271.5 + 124) * np.sin(5*np.pi / 3), center3[0,1] ], z=[0, center3[0,2]],
        mode='lines+markers',
        marker=dict(size=5, color='blue'),
        line=dict(color='blue', width=5),
        name='Vector to Arm 3'
    ))

    # Add Intersection Points
    if len(intersection_points_cyl1_2) > 0:
        fig.add_trace(go.Scatter3d(
            x=intersection_points_cyl1_2[:, 0], y=intersection_points_cyl1_2[:, 1], z=intersection_points_cyl1_2[:, 2],
            mode='markers', marker=dict(size=4, color='green', opacity=0.8), name=f'Intersections'
        ))

    # Add Intersection Points
    if len(intersection_points_cyl1_3) > 0:
        fig.add_trace(go.Scatter3d(
            x=intersection_points_cyl1_3[:, 0], y=intersection_points_cyl1_3[:, 1], z=intersection_points_cyl1_3[:, 2],
            mode='markers', marker=dict(size=4, color='green', opacity=0.8), name=f'Intersections'
        ))


    # Add Min Positive and Negative Z Points for Cylinder 1 with 2
    if min_cyl1_2_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[min_cyl1_2_inertial[0,0]], y=[min_cyl1_2_inertial[0,1]], z=[min_cyl1_2_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='cyan'), name=f'Min Cylinder 1 with 2, Distance={min_distance_cyl_1_2:.2f}'
        ))
    if max_cyl1_2_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[max_cyl1_2_inertial[0,0]], y=[max_cyl1_2_inertial[0,1]], z=[max_cyl1_2_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='blue'), name=f'Max Cylinder 1 with 2, Distance={max_distance_cyl_1_2:.2f}'
        ))

    # Add Min Positive and Negative Z Points for Cylinder 2 with 1
    if min_cyl2_1_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[min_cyl2_1_inertial[0,0]], y=[min_cyl2_1_inertial[0,1]], z=[min_cyl2_1_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='magenta'), name=f'Min Cylinder 2 with 1, Distance={min_distance_cyl_2_1:.2f}'
        ))
    if max_cyl2_1_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[max_cyl2_1_inertial[0,0]], y=[max_cyl2_1_inertial[0,1]], z=[max_cyl2_1_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='purple'), name=f'Max Cylinder 2 with 1, Distance={max_distance_cyl_2_1:.2f}'
        ))

    # Add Min Positive and Negative Z Points for Cylinder 1 with 3
    if min_cyl1_3_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[min_cyl1_3_inertial[0,0]], y=[min_cyl1_3_inertial[0,1]], z=[min_cyl1_3_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='cyan'), name=f'Min Cylinder 1 with 3, Distance={min_distance_cyl_1_3:.2f}'
        ))
    if max_cyl1_3_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[max_cyl1_3_inertial[0,0]], y=[max_cyl1_3_inertial[0,1]], z=[max_cyl1_3_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='blue'), name=f'Max Cylinder 1 with 3, Distance={max_distance_cyl_1_3:.2f}'
        ))

    # Add Min Positive and Negative Z Points for Cylinder 3 with 1
    if min_cyl3_1_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[min_cyl3_1_inertial[0,0]], y=[min_cyl3_1_inertial[0,1]], z=[min_cyl3_1_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='magenta'), name=f'Min Cylinder 3 with 1, Distance={min_distance_cyl_3_1:.2f}'
        ))
    if max_cyl3_1_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[max_cyl3_1_inertial[0,0]], y=[max_cyl3_1_inertial[0,1]], z=[max_cyl3_1_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='purple'), name=f'Max Cylinder 3 with 1, Distance={max_distance_cyl_3_1:.2f}'
        ))

    # Customize layout
    fig.update_layout(
        title=f"Cylinder Intersection and Closest Points (Meausure={np.degrees(a1):.1f}°, Left={np.degrees(a2):.1f}°, Right={np.degrees(a3):.1f}°)",
        scene=dict(
            aspectmode='cube',
            xaxis=dict(title="X-axis", range=[-1000, 1000]),
            yaxis=dict(title="Y-axis", range=[-1000, 1000]),
            zaxis=dict(title="Z-axis", range=[-1000, 1000]),
        ),
        margin=dict(l=0, r=0, t=40, b=0),
        showlegend=True
    )

    # Show plot
    fig.show()


# Define parameters
threshold = 1
radius_prop = 304.8/2
armlength = 271.5 + 124
z_offset = -52.7
#z_offset = 0

# Normalised with prop radius
z_sections = [0.5, 0, -0.9, -1.6, -4.5]
z_sections = [1, 0, -1]
z_sections = sorted(z_sections, reverse=False)

resolution_theta = int(np.round(2*np.pi*radius_prop/threshold/1.4))
resolution_z = int(np.round(2*radius_prop*(max(z_sections) - min(z_sections))/threshold/1.4))
#print(resolution_theta)
#print(resolution_z)


data = []

# Define angles to iterate over
A1 = np.linspace(0, 2 * np.pi, 6, endpoint=False)
A2 = np.linspace(0, 2 * np.pi, 6, endpoint=False)
A1 = np.linspace(0, 2 * np.pi, 4, endpoint=False)
A2 = np.linspace(0, 2 * np.pi, 4, endpoint=False)
A3 = np.linspace(0, 2 * np.pi, 4, endpoint=False)
#A1 = [np.radians(60)]
A3 = [np.radians(60)]
#angList = list(itertools.product(A1, A2, A3))
#angList = [(np.radians(0),  np.radians(270), np.radians(90))]
angList = [(np.radians(260), np.radians(220), np.radians(210)), (np.radians(80), np.radians(30), np.radians(40))]
#angList = [(np.radians(210), np.radians(210)), (np.radians(210), np.radians(240)), (np.radians(210), np.radians(270))]
#print(f"Angle from `angList` using linspace: {angList[92]}")
#print(f"Manually input angle (210, 240): {(np.radians(210), np.radians(240))}")
#
#print("Difference in angles:")
#print(np.array(angList[92]) - np.array((np.radians(210), np.radians(240))))
#print("Are they close?:", np.isclose(angList[92], (np.radians(210), np.radians(240))))
#print(angList)

angList = [(np.radians(0), np.radians(-90), np.radians(90))]

angList = [(np.round(a[0], 8), np.round(a[1], 8), np.round(a[2], 8)) for a in angList]
# Initialize variables to track the previous state of a1 and cached data
previous_a1 = None
cylinder1_points = None
cylinder1_kdtree = None
cylinder1_surface = None

previous_a2 = None
cylinder2_points = None
cylinder2_surface = None

# Iterate through each combination of angles
for a1, a2, a3 in angList:
    # Check if a1 has changed
    if a2 != previous_a2 or a1 != previous_a1:
        if a1 != previous_a1:
            # Update cylinder1 and its associated data only when a1 changes
            TI1, _, _ = create_transformation_matrix(a1, a2, a3)  # Only need TI1 for cylinder1
            T1I = invert_transformation(TI1)
            origin_point = np.zeros((1, 3))
            center1 = apply_transform(TI1, origin_point)

            # Generate and transform Cylinder 1
            cylinder1_points = generate_slipstream(
                radius_prop, z_sections, radius_funcs, resolution_theta=resolution_theta, resolution_z=resolution_z
            )
            cylinder1_points = apply_transform(TI1, cylinder1_points)

            # Build KDTree for Cylinder 1
            cylinder1_kdtree = KDTree(cylinder1_points)

            # Generate surface grids for Cylinder 1 (downsample for plotting)
            cylinder1_surface = generate_surface_grids(cylinder1_points, resolution_theta, resolution_z)
            cylinder1_surface = tuple(downsample_grid(g, factor=5) for g in cylinder1_surface)

            # Update the cached `previous_a1`
            previous_a1 = a1

        # Process Cylinder 2 for the current a2
        _, TI2, _ = create_transformation_matrix(a1, a2, a3)
        T2I = invert_transformation(TI2)
        origin_point = np.zeros((1, 3))
        center2 = apply_transform(TI2, origin_point)

        # Generate and transform Cylinder 2
        cylinder2_points = generate_slipstream(
            radius_prop, z_sections, radius_funcs, resolution_theta=resolution_theta, resolution_z=resolution_z
        )
        #print(cylinder2_points.shape)
        cylinder2_points = apply_transform(TI2, cylinder2_points)
        #print(cylinder2_points.shape)
        (intersection_points_cyl1_2,
        min_distance_cyl_1_2, max_distance_cyl_1_2,
        min_distance_cyl_2_1, max_distance_cyl_2_1,
        min_cyl1_2_inertial, max_cyl1_2_inertial, 
        min_cyl2_1_inertial, max_cyl2_1_inertial) = find_intersection_points(
        cylinder1_kdtree, cylinder1_points, cylinder2_points, TI1, TI2, T1I, T2I, threshold)

        # Generate surface grids for Cylinder 2 (downsample for plotting)
        cylinder2_surface = generate_surface_grids(cylinder2_points, resolution_theta, resolution_z)
        cylinder2_surface = tuple(downsample_grid(g, factor=5) for g in cylinder2_surface)

        previous_a2 = a2

    _, _, TI3 = create_transformation_matrix(a1, a2, a3)
    cylinder3_points = generate_slipstream(radius_prop, z_sections, radius_funcs, resolution_theta, resolution_z)
    cylinder3_points = apply_transform(TI3, cylinder3_points)
    T3I = invert_transformation(TI3)
    origin_point = np.zeros((1, 3))
    center3 = apply_transform(TI3, origin_point)
    # Find intersection points using precomputed KDTree for Cylinder 1
    (intersection_points_cyl1_3,
    min_distance_cyl_1_3, max_distance_cyl_1_3,
    min_distance_cyl_3_1, max_distance_cyl_3_1,
    min_cyl1_3_inertial, max_cyl1_3_inertial, 
    min_cyl3_1_inertial, max_cyl3_1_inertial) = find_intersection_points(
    cylinder1_kdtree, cylinder1_points, cylinder3_points, TI1, TI3, T1I, T3I, threshold)

    # Generate surface grids for Cylinder 3 (downsample for plotting)
    cylinder3_surface = generate_surface_grids(cylinder3_points, resolution_theta, resolution_z)
    #cylinder3_surface = tuple(downsample_grid(g, factor=5) for g in cylinder3_surface)

    # Plot cylinders and intersection points
    plot_cylinders(cylinder1_surface, cylinder2_surface, cylinder3_surface,
                    intersection_points_cyl1_2, intersection_points_cyl1_3,
                    min_distance_cyl_1_2, max_distance_cyl_1_2,
                    min_distance_cyl_2_1, max_distance_cyl_2_1,
                    min_distance_cyl_1_3, max_distance_cyl_1_3,
                    min_distance_cyl_3_1, max_distance_cyl_3_1, 
                    min_cyl1_2_inertial, max_cyl1_2_inertial, 
                    min_cyl2_1_inertial, max_cyl2_1_inertial,
                    min_cyl1_3_inertial, max_cyl1_3_inertial, 
                    min_cyl3_1_inertial, max_cyl3_1_inertial,
                    center1, center2, center3, a1, a2, a3)

    # Store results in the dataframe
    data.append([np.degrees(a1), np.degrees(a2), np.degrees(a3), min_distance_cyl_1_2, max_distance_cyl_1_2,
                    min_distance_cyl_2_1, max_distance_cyl_2_1,
                    min_distance_cyl_1_3, max_distance_cyl_1_3,
                    min_distance_cyl_3_1, max_distance_cyl_3_1])
            

# Create the DataFrame from collected data
#df = pd.DataFrame(data, columns=['a1', 'a2', 'a3','min_distance_cyl_1_2', 'max_distance_cyl_1_2',
#                    'min_distance_cyl_2_1', 'max_distance_cyl_2_1',
#                    'min_distance_cyl_1_3', 'max_distance_cyl_1_3',
#                    'min_distance_cyl_3_1', 'max_distance_cyl_3_1'])
#print(df.shape)
##df.replace([np.inf, -np.inf], np.nan, inplace=True)
##df = df.dropna()
##print(df.shape)
#print(df.round().to_string())


In [ ]:
A1_1 = np.linspace(0, np.pi/2, 7)
A1_2 = np.linspace(np.pi, 3*np.pi/2, 7)
A1 = np.concatenate((A1_1, A1_2))
A1 = np.linspace(0, 2 * np.pi, 36, endpoint=False)
A2 = np.linspace(0, 2 * np.pi, 36, endpoint=False)
A3 = np.linspace(0, 2 * np.pi, 36, endpoint=False)
#A1 = [np.radians(60)]
angList = list(itertools.product(A1, A2, A3))
print(len(angList))
column_names = ['a1', 'a2_min', 'a2_max']
df_a2 = pd.read_csv('./angular_ranges_1_2.csv', sep=';', names=column_names, header=None)
df_a2 = pd.read_csv('./angular_ranges_sym_1_2.csv', sep=';', names=column_names, header=None)
# Convert a2_max
df_a2['a2_max'] = df_a2.apply(lambda row: row['a2_max'] + 360 if row['a2_max'] < row['a2_min'] else row['a2_max'], axis=1)
column_names = ['a1', 'a3_min', 'a3_max']
df_a3 = pd.read_csv('./angular_ranges_1_3.csv', sep=';', names=column_names, header=None)
df_a3 = pd.read_csv('./angular_ranges_sym_1_3.csv', sep=';', names=column_names, header=None)
df_a3['a3_min'] = df_a3.apply(lambda row: row['a3_min'] - 360 if row['a3_max'] < row['a3_min'] else row['a3_min'], axis=1)

valid_combinations = []

for a1, a2, a3 in angList:
    a1_deg, a2_deg, a3_deg = np.round(np.degrees(a1)), np.round(np.degrees(a2)), np.round(np.degrees(a3))
    #if not (90 <= a1_deg <= 180 or 270 <= a1_deg <= 360):
    #    continue  # Skip this combination if a1 is not in the specified range

    #if not (90 <= a1_deg <= 180 or 270 <= a1_deg <= 360):
    #    continue  # Skip this combination if a1 is not in the specified range

    a2_row = df_a2[df_a2['a1'] == a1_deg]
    a3_row = df_a3[df_a3['a1'] == a1_deg]
    #print(a2_row)
    if not a2_row.empty and not a3_row.empty:
        #print('hello')
        a2_min, a2_max = a2_row.iloc[0]['a2_min'], a2_row.iloc[0]['a2_max']
        a3_min, a3_max = a3_row.iloc[0]['a3_min'], a3_row.iloc[0]['a3_max']
        #print(a1)
        #print(a2_min, a2_max)
        #print(a3_min, a3_max)
        # Normalize a2 and a3 to handle wrap-around
        a2_norm = a2_deg + 360 if a2_deg < a2_min else a2_deg
        a3_norm = a3_deg - 360 if a3_deg > a3_max else a3_deg
        #if a1_deg == 270:
            #print(a3_deg, a3_norm, a3_min, a3_max)
        #print(a1_deg, a3_deg, a3_norm)
        if a2_min <= a2_norm <= a2_max and a3_min <= a3_norm <= a3_max:
            valid_combinations.append((a1, a2, a3))
            #print(a1_deg, a2_deg, a3_deg)

# Print valid combinations
#print("Valid angle combinations:", valid_combinations)
print(len(valid_combinations))
angList = valid_combinations

In [ ]:
import numpy as np
import plotly.graph_objects as go
import itertools
import pandas as pd
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
# Function to generate a cylinder with variable radius

def generate_slipstream(rad, z_sections, radius_funcs, resolution_theta=50, resolution_z=50):
    z_min, z_max = min(z_sections), max(z_sections)
    z_steps = np.linspace(z_min, z_max, resolution_z)
    theta = np.linspace(0, 2 * np.pi, resolution_theta)

    radius = np.zeros_like(z_steps)
    for i in range(len(z_sections) - 1):
        mask = (z_steps >= z_sections[i]) & (z_steps <= z_sections[i + 1])
        radius[mask] = rad * radius_funcs[i](z_steps[mask])

    # Avoid creating full grids
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    x = radius[:, None] * cos_theta
    y = radius[:, None] * sin_theta
    z = (z_steps * rad * 2)[:, None] + np.zeros_like(theta)

    # Combine points efficiently
    points = np.column_stack((x.ravel(), y.ravel(), z.ravel()))
    return points


# Define radius functions
#def radius_above(z): return 1 + z *0.25
#def radius_contract(z): return 1 + 0*z
#def radius_nf(z): return 1 - (z+0.9)*0.4
#def radius_ff(z): return 1.3 - (z+1.6)*0.3
#radius_funcs = [radius_ff, radius_nf, radius_contract, radius_above]

def radius_above(z): return 1 +z*0
def radius_contract(z): return 1 + z*0
radius_funcs = [radius_above, radius_contract]

# Function to generate surface grids for plotting
def generate_surface_grids(points, resolution_theta, resolution_z):
    x, y, z = points[:, 0], points[:, 1], points[:, 2]
    x_grid = x.reshape(resolution_z, resolution_theta)
    y_grid = y.reshape(resolution_z, resolution_theta)
    z_grid = z.reshape(resolution_z, resolution_theta)
    return x_grid, y_grid, z_grid


# Function to create rotation matrices for given angles
def create_transformation_matrix(a1, a2, a3):
    TI1_1 = np.array([
        [1, 0, 0, 0],
        [0, np.cos(a1), -np.sin(a1), 0],
        [0, np.sin(a1), np.cos(a1), 0],
        [0, 0, 0, 1]
    ])
    T1_2 = np.array([
        [1, 0, 0, armlength],
        [0, 1, 0, 0],
        [0, 0, 1, z_offset],
        [0, 0, 0, 1]
    ])

    TI2_1 = np.array([
        [np.cos(np.pi / 3), -np.sin(np.pi / 3), 0, 0],
        [np.sin(np.pi / 3), np.cos(np.pi / 3), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    T2_2 = np.array([
        [1, 0, 0, armlength],
        [0, np.cos(a2), -np.sin(a2), 0],
        [0, np.sin(a2), np.cos(a2), 0],
        [0, 0, 0, 1]
    ])
    TI2_3 = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, z_offset],
        [0, 0, 0, 1]
    ])

    TI3_1 = np.array([
        [np.cos(-np.pi / 3), -np.sin(-np.pi / 3), 0, 0],
        [np.sin(-np.pi / 3), np.cos(-np.pi / 3), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    T3_2 = np.array([
        [1, 0, 0, armlength],
        [0, np.cos(a3), -np.sin(a3), 0],
        [0, np.sin(a3), np.cos(a3), 0],
        [0, 0, 0, 1]
    ])
    TI3_3 = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, z_offset],
        [0, 0, 0, 1]
    ])


    TI1 = np.matmul(TI1_1,T1_2)
    TI2 = np.matmul(np.matmul(TI2_1, T2_2), TI2_3)
    TI3 = np.matmul(np.matmul(TI3_1, T3_2), TI3_3)
    return TI1, TI2, TI3

def invert_transformation(matrix):
    """
    Computes the inverse of a 4x4 homogeneous transformation matrix.
    
    Args:
        T (np.ndarray): A 4x4 homogeneous transformation matrix.
    
    Returns:
        np.ndarray: The inverse of the input transformation matrix.
    """
    # Extract rotation matrix (3x3) and translation vector (3x1)
    R = matrix[:3, :3]
    t = matrix[:3, 3]
    
    # Inverse of rotation is just the transpose of R
    R_inv = R.T
    
    # Inverse of translation is -R_inv * t
    t_inv = -np.dot(R_inv, t)
    
    # Construct the inverse transformation matrix
    matrix_inv = np.eye(4)
    matrix_inv[:3, :3] = R_inv
    matrix_inv[:3, 3] = t_inv
    
    return matrix_inv


# Transformation matrix
def apply_transform(matrix, points):
    num_points = points.shape[0]
    homogeneous_points = np.c_[points, np.ones(num_points)]  # Inline addition of the homogeneous coordinate
    return np.dot(homogeneous_points, matrix.T)[:, :3]


def find_intersection_points(cylinder1_kdtree, cylinder1_points, cylinder2_points, ti1, ti2, t1i, t2i, threshold=0.1):
    """
    Finds intersection points between two cylinders, transforms them back, and returns the closest Z points.
    
    Args:
        cylinder1_kdtree: KDTree for Cylinder 1.
        cylinder1_points: Points of Cylinder 1 in its local frame.
        cylinder2_points: Points of Cylinder 2 in its local frame.
        ti1_inv: Inverse transformation matrix for Cylinder 1.
        ti2_inv: Inverse transformation matrix for Cylinder 2.
        threshold: Distance threshold to consider points intersecting.
    
    Returns:
        - Transformed intersection points back into the inertial frame.
        - Closest point with minimum +Z and -Z for both cylinders.
        - Z values and coordinates in the inertial frame.
    """

    # Step 1: Query KDTree for intersections within a threshold
    distances, indices = cylinder1_kdtree.query(cylinder2_points, distance_upper_bound=threshold)
    valid_mask = distances < threshold
    intersection_points_cyl2 = cylinder2_points[valid_mask]

    # Step 2: Transform intersection points back to Cylinder 1 and 2 local coordinates
    intersection_points_cyl1 = cylinder1_points[indices[valid_mask]]
    if len(intersection_points_cyl1) == 0:
        return [], None, None, None, None, None, None, None, None

    # Transform using inverse matrices to local cylinder coordinates
    intersection_points_cyl1_transformed = apply_transform(t1i,intersection_points_cyl1)
    intersection_points_cyl2_transformed = apply_transform(t2i,intersection_points_cyl2)
    # Step 3: Find min +Z and -Z values and corresponding points
    #print(intersection_points_cyl1)
    #print(intersection_points_cyl1_transformed)

    # Initialize the result variables in case no valid points are found
    min_cyl1 = max_cyl1 = None
    min_cyl1_idx = max_cyl1_idx = None
    min_cyl2 = max_cyl2 = None
    min_cyl2_idx = max_cyl2_idx = None
    min_distance_cyl_1 = max_distance_cyl_1 = min_distance_cyl_2 = max_distance_cyl_2 = None


    if np.any(intersection_points_cyl1_transformed):
        min_cyl1_idx = np.argmin(intersection_points_cyl1_transformed[:, 2])
        min_cyl1 = intersection_points_cyl1_transformed[min_cyl1_idx].reshape(1, 3)
        min_distance_cyl_1 = min_cyl1[0, 2]

        max_cyl1_idx = np.argmax(intersection_points_cyl1_transformed[:, 2])
        max_cyl1 = intersection_points_cyl1_transformed[max_cyl1_idx].reshape(1, 3)
        max_distance_cyl_1 = max_cyl1[0, 2]


    if np.any(intersection_points_cyl2_transformed):
        min_cyl2_idx = np.argmin(intersection_points_cyl2_transformed[:, 2])
        min_cyl2 = intersection_points_cyl2_transformed[min_cyl2_idx].reshape(1, 3)
        min_distance_cyl_2 = min_cyl2[0, 2]

        max_cyl2_idx = np.argmax(intersection_points_cyl2_transformed[:, 2])
        max_cyl2 = intersection_points_cyl2_transformed[max_cyl2_idx].reshape(1, 3)
        max_distance_cyl_2 = max_cyl2[0, 2]




    # Step 3: Transform the points back to the inertial frame, if they were found
    min_cyl1_inertial = apply_transform(ti1, min_cyl1) if min_cyl1 is not None else None
    #print(min_negative_z_cyl1.shape)
    #print(min_negative_z_cyl1)
    max_cyl1_inertial = apply_transform(ti1, max_cyl1) if max_cyl1 is not None else None
    min_cyl2_inertial = apply_transform(ti2, min_cyl2) if min_cyl2 is not None else None
    max_cyl2_inertial = apply_transform(ti2, max_cyl2) if max_cyl2 is not None else None

    return (
        intersection_points_cyl1,
        min_distance_cyl_1,
        max_distance_cyl_1,
        min_distance_cyl_2,
        max_distance_cyl_2,
        min_cyl1_inertial,
        max_cyl1_inertial,
        min_cyl2_inertial,
        max_cyl2_inertial
    )

def downsample_grid(grid, factor):
    return grid[::factor, ::factor]


# Function to plot the cylinders, intersection points, and closest points
def plot_cylinders(cylinder1_surface, cylinder2_surface, cylinder3_surface,
                    intersection_points_cyl1_2, intersection_points_cyl1_3,
                    min_distance_cyl_1_2, max_distance_cyl_1_2,
                    min_distance_cyl_2_1, max_distance_cyl_2_1,
                    min_distance_cyl_1_3, max_distance_cyl_1_3,
                    min_distance_cyl_3_1, max_distance_cyl_3_1, 
                    min_cyl1_2_inertial, max_cyl1_2_inertial, 
                    min_cyl2_1_inertial, max_cyl2_1_inertial,
                    min_cyl1_3_inertial, max_cyl1_3_inertial, 
                    min_cyl3_1_inertial, max_cyl3_1_inertial,
                    center1, center2, center3, a1, a2, a3):
    fig = go.Figure()

    # Add Cylinder 1 as a surface
    fig.add_trace(go.Surface(
        x=cylinder1_surface[0], y=cylinder1_surface[1], z=cylinder1_surface[2],
        colorscale='Greens',
        opacity=0.7,
        showscale=False,
        name=f'Cylinder 1'
    ))

    # Add Cylinder 2 as a surface
    fig.add_trace(go.Surface(
        x=cylinder2_surface[0], y=cylinder2_surface[1], z=cylinder2_surface[2],
        colorscale='Reds',
        opacity=0.7,
        showscale=False,
        name=f'Cylinder 2'
    ))

    # Add Cylinder 3 as a surface
    fig.add_trace(go.Surface(
        x=cylinder3_surface[0], y=cylinder3_surface[1], z=cylinder3_surface[2],
        colorscale='Blues',
        opacity=0.7,
        showscale=False,
        name=f'Cylinder 3'
    ))

    # Add vector from origin to Cylinder 1 center
    fig.add_trace(go.Scatter3d(
        x=[0, 271.5 + 124], y=[0, 0], z=[0, 0],
        mode='lines+markers',
        marker=dict(size=5, color='green'),
        line=dict(color='green', width=5),
        name='Vector to Arm 1'
    ))
    fig.add_trace(go.Scatter3d(
        x=[271.5 + 124, center1[0,0]], y=[0, center1[0,1]], z=[0, center1[0,2]],
        mode='lines+markers',
        marker=dict(size=5, color='green'),
        line=dict(color='green', width=5),
        name='Vector to Arm 1'
    ))

    # Add vector from origin to Cylinder 2 center
    fig.add_trace(go.Scatter3d(
        x=[0, (271.5 + 124) * np.cos(np.pi / 3)], y=[0, (271.5 + 124) * np.sin(np.pi / 3)], z=[0, 0],
        mode='lines+markers',
        marker=dict(size=5, color='red'),
        line=dict(color='red', width=5),
        name='Vector to Arm 2'
    ))
    fig.add_trace(go.Scatter3d(
        x=[(271.5 + 124) * np.cos(np.pi / 3), center2[0,0]], y=[(271.5 + 124) * np.sin(np.pi / 3), center2[0,1] ], z=[0, center2[0,2]],
        mode='lines+markers',
        marker=dict(size=5, color='red'),
        line=dict(color='red', width=5),
        name='Vector to Arm 2'
    ))

    # Add vector from origin to Cylinder 3 center
    fig.add_trace(go.Scatter3d(
        x=[0, (271.5 + 124) * np.cos(5*np.pi / 3)], y=[0, (271.5 + 124) * np.sin(5*np.pi / 3)], z=[0, 0],
        mode='lines+markers',
        marker=dict(size=5, color='blue'),
        line=dict(color='blue', width=5),
        name='Vector to Arm 3'
    ))
    fig.add_trace(go.Scatter3d(
        x=[(271.5 + 124) * np.cos(5*np.pi / 3), center3[0,0]], y=[(271.5 + 124) * np.sin(5*np.pi / 3), center3[0,1] ], z=[0, center3[0,2]],
        mode='lines+markers',
        marker=dict(size=5, color='blue'),
        line=dict(color='blue', width=5),
        name='Vector to Arm 3'
    ))

    # Add Intersection Points
    if len(intersection_points_cyl1_2) > 0:
        fig.add_trace(go.Scatter3d(
            x=intersection_points_cyl1_2[:, 0], y=intersection_points_cyl1_2[:, 1], z=intersection_points_cyl1_2[:, 2],
            mode='markers', marker=dict(size=4, color='green', opacity=0.8), name=f'Intersections'
        ))

    # Add Intersection Points
    if len(intersection_points_cyl1_3) > 0:
        fig.add_trace(go.Scatter3d(
            x=intersection_points_cyl1_3[:, 0], y=intersection_points_cyl1_3[:, 1], z=intersection_points_cyl1_3[:, 2],
            mode='markers', marker=dict(size=4, color='green', opacity=0.8), name=f'Intersections'
        ))


    # Add Min Positive and Negative Z Points for Cylinder 1 with 2
    if min_cyl1_2_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[min_cyl1_2_inertial[0,0]], y=[min_cyl1_2_inertial[0,1]], z=[min_cyl1_2_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='cyan'), name=f'Min Cylinder 1 with 2, Distance={min_distance_cyl_1_2:.2f}'
        ))
    if max_cyl1_2_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[max_cyl1_2_inertial[0,0]], y=[max_cyl1_2_inertial[0,1]], z=[max_cyl1_2_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='blue'), name=f'Max Cylinder 1 with 2, Distance={max_distance_cyl_1_2:.2f}'
        ))

    # Add Min Positive and Negative Z Points for Cylinder 2 with 1
    if min_cyl2_1_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[min_cyl2_1_inertial[0,0]], y=[min_cyl2_1_inertial[0,1]], z=[min_cyl2_1_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='magenta'), name=f'Min Cylinder 2 with 1, Distance={min_distance_cyl_2_1:.2f}'
        ))
    if max_cyl2_1_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[max_cyl2_1_inertial[0,0]], y=[max_cyl2_1_inertial[0,1]], z=[max_cyl2_1_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='purple'), name=f'Max Cylinder 2 with 1, Distance={max_distance_cyl_2_1:.2f}'
        ))

    # Add Min Positive and Negative Z Points for Cylinder 1 with 3
    if min_cyl1_3_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[min_cyl1_3_inertial[0,0]], y=[min_cyl1_3_inertial[0,1]], z=[min_cyl1_3_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='cyan'), name=f'Min Cylinder 1 with 3, Distance={min_distance_cyl_1_3:.2f}'
        ))
    if max_cyl1_3_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[max_cyl1_3_inertial[0,0]], y=[max_cyl1_3_inertial[0,1]], z=[max_cyl1_3_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='blue'), name=f'Max Cylinder 1 with 3, Distance={max_distance_cyl_1_3:.2f}'
        ))

    # Add Min Positive and Negative Z Points for Cylinder 3 with 1
    if min_cyl3_1_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[min_cyl3_1_inertial[0,0]], y=[min_cyl3_1_inertial[0,1]], z=[min_cyl3_1_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='magenta'), name=f'Min Cylinder 3 with 1, Distance={min_distance_cyl_3_1:.2f}'
        ))
    if max_cyl3_1_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[max_cyl3_1_inertial[0,0]], y=[max_cyl3_1_inertial[0,1]], z=[max_cyl3_1_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='purple'), name=f'Max Cylinder 3 with 1, Distance={max_distance_cyl_3_1:.2f}'
        ))

    # Customize layout
    fig.update_layout(
        title=f"Cylinder Intersection and Closest Points (a1={np.degrees(a1):.1f}°, a2={np.degrees(a2):.1f}°, a3={np.degrees(a3):.1f}°)",
        scene=dict(
            aspectmode='cube',
            xaxis=dict(title="X-axis", range=[-1000, 1000]),
            yaxis=dict(title="Y-axis", range=[-1000, 1000]),
            zaxis=dict(title="Z-axis", range=[-1000, 1000]),
        ),
        margin=dict(l=0, r=0, t=40, b=0),
        showlegend=True
    )

    # Show plot
    fig.show()


# Define parameters
threshold = 1
radius_prop = 304.8/2
armlength = 271.5 + 124
z_offset = -52.7
#z_offset = 0

# Normalised with prop radius
z_sections = [0.5, 0, -0.9, -1.6, -4.5]
z_sections = [1, 0, -1]
z_sections = sorted(z_sections, reverse=False)

resolution_theta = int(np.round(2*np.pi*radius_prop/threshold/1.4))
resolution_z = int(np.round(2*radius_prop*(max(z_sections) - min(z_sections))/threshold/1.4))
#print(resolution_theta)
#print(resolution_z)


data = []

# Define angles to iterate over
A1 = np.linspace(0, 2 * np.pi, 6, endpoint=False)
A2 = np.linspace(0, 2 * np.pi, 6, endpoint=False)
A1 = np.linspace(0, 2 * np.pi, 4, endpoint=False)
A2 = np.linspace(0, 2 * np.pi, 4, endpoint=False)
A3 = np.linspace(0, 2 * np.pi, 4, endpoint=False)
#A1 = [np.radians(60)]
A3 = [np.radians(60)]
#angList = list(itertools.product(A1, A2, A3))
#angList = [(np.radians(180),  np.radians(270), np.radians(90))]
#angList = [(np.radians(210), np.radians(210)), (np.radians(210), np.radians(240)), (np.radians(210), np.radians(270)), (np.radians(330), np.radians(300))]
#angList = [(np.radians(210), np.radians(210)), (np.radians(210), np.radians(240)), (np.radians(210), np.radians(270))]
#print(f"Angle from `angList` using linspace: {angList[92]}")
#print(f"Manually input angle (210, 240): {(np.radians(210), np.radians(240))}")
#
#print("Difference in angles:")
#print(np.array(angList[92]) - np.array((np.radians(210), np.radians(240))))
#print("Are they close?:", np.isclose(angList[92], (np.radians(210), np.radians(240))))
#print(angList)

angList = [(np.round(a[0], 8), np.round(a[1], 8), np.round(a[2], 8)) for a in angList]
# Initialize variables to track the previous state of a1 and cached data
previous_a1 = None
cylinder1_points = None
cylinder1_kdtree = None
cylinder1_surface = None

previous_a2 = None
cylinder2_points = None
cylinder2_surface = None

# Iterate through each combination of angles
for a1, a2, a3 in angList:
    # Check if a1 has changed
    if a2 != previous_a2 or a1 != previous_a1:
        if a1 != previous_a1:
            # Update cylinder1 and its associated data only when a1 changes
            TI1, _, _ = create_transformation_matrix(a1, a2, a3)  # Only need TI1 for cylinder1
            T1I = invert_transformation(TI1)
            origin_point = np.zeros((1, 3))
            center1 = apply_transform(TI1, origin_point)

            # Generate and transform Cylinder 1
            cylinder1_points = generate_slipstream(
                radius_prop, z_sections, radius_funcs, resolution_theta=resolution_theta, resolution_z=resolution_z
            )
            cylinder1_points = apply_transform(TI1, cylinder1_points)

            # Build KDTree for Cylinder 1
            cylinder1_kdtree = KDTree(cylinder1_points)

            # Generate surface grids for Cylinder 1 (downsample for plotting)
            #cylinder1_surface = generate_surface_grids(cylinder1_points, resolution_theta, resolution_z)
            #cylinder1_surface = tuple(downsample_grid(g, factor=5) for g in cylinder1_surface)

            # Update the cached `previous_a1`
            previous_a1 = a1

        # Process Cylinder 2 for the current a2
        _, TI2, _ = create_transformation_matrix(a1, a2, a3)
        T2I = invert_transformation(TI2)
        origin_point = np.zeros((1, 3))
        center2 = apply_transform(TI2, origin_point)

        # Generate and transform Cylinder 2
        cylinder2_points = generate_slipstream(
            radius_prop, z_sections, radius_funcs, resolution_theta=resolution_theta, resolution_z=resolution_z
        )
        #print(cylinder2_points.shape)
        cylinder2_points = apply_transform(TI2, cylinder2_points)
        #print(cylinder2_points.shape)
        (intersection_points_cyl1_2,
        min_distance_cyl_1_2, max_distance_cyl_1_2,
        min_distance_cyl_2_1, max_distance_cyl_2_1,
        min_cyl1_2_inertial, max_cyl1_2_inertial, 
        min_cyl2_1_inertial, max_cyl2_1_inertial) = find_intersection_points(
        cylinder1_kdtree, cylinder1_points, cylinder2_points, TI1, TI2, T1I, T2I, threshold)

        # Generate surface grids for Cylinder 2 (downsample for plotting)
        #cylinder2_surface = generate_surface_grids(cylinder2_points, resolution_theta, resolution_z)
        #cylinder2_surface = tuple(downsample_grid(g, factor=5) for g in cylinder2_surface)

        previous_a2 = a2

    _, _, TI3 = create_transformation_matrix(a1, a2, a3)
    cylinder3_points = generate_slipstream(radius_prop, z_sections, radius_funcs, resolution_theta, resolution_z)
    cylinder3_points = apply_transform(TI3, cylinder3_points)
    T3I = invert_transformation(TI3)
    origin_point = np.zeros((1, 3))
    center3 = apply_transform(TI3, origin_point)
    # Find intersection points using precomputed KDTree for Cylinder 1
    (intersection_points_cyl1_3,
    min_distance_cyl_1_3, max_distance_cyl_1_3,
    min_distance_cyl_3_1, max_distance_cyl_3_1,
    min_cyl1_3_inertial, max_cyl1_3_inertial, 
    min_cyl3_1_inertial, max_cyl3_1_inertial
    ) = find_intersection_points(
    cylinder1_kdtree, cylinder1_points, cylinder3_points, TI1, TI3, T1I, T3I, threshold)

    # Generate surface grids for Cylinder 3 (downsample for plotting)
    #cylinder3_surface = generate_surface_grids(cylinder3_points, resolution_theta, resolution_z)
    #cylinder3_surface = tuple(downsample_grid(g, factor=5) for g in cylinder3_surface)

    # Plot cylinders and intersection points
    #plot_cylinders(cylinder1_surface, cylinder2_surface, cylinder3_surface,
    #                intersection_points_cyl1_2, intersection_points_cyl1_3,
    #                min_distance_cyl_1_2, max_distance_cyl_1_2,
    #                min_distance_cyl_2_1, max_distance_cyl_2_1,
    #                min_distance_cyl_1_3, max_distance_cyl_1_3,
    #                min_distance_cyl_3_1, max_distance_cyl_3_1, 
    #                min_cyl1_2_inertial, max_cyl1_2_inertial, 
    #                min_cyl2_1_inertial, max_cyl2_1_inertial,
    #                min_cyl1_3_inertial, max_cyl1_3_inertial, 
    #                min_cyl3_1_inertial, max_cyl3_1_inertial,
    #                center1, center2, center3, a1, a2, a3)

    # Store results in the dataframe
    data.append([np.degrees(a1), np.degrees(a2), np.degrees(a3), min_distance_cyl_1_2, max_distance_cyl_1_2,
                    min_distance_cyl_2_1, max_distance_cyl_2_1,
                    min_distance_cyl_1_3, max_distance_cyl_1_3,
                    min_distance_cyl_3_1, max_distance_cyl_3_1])
            

# Create the DataFrame from collected data
df = pd.DataFrame(data, columns=['a1', 'a2', 'a3','min_distance_cyl_1_2', 'max_distance_cyl_1_2',
                    'min_distance_cyl_2_1', 'max_distance_cyl_2_1',
                    'min_distance_cyl_1_3', 'max_distance_cyl_1_3',
                    'min_distance_cyl_3_1', 'max_distance_cyl_3_1'])
print(df.shape)
#df.replace([np.inf, -np.inf], np.nan, inplace=True)
#df = df.dropna()
#print(df.shape)
print(df.round().to_string())


In [ ]:
df_cleaned = df.copy()
df_cleaned = df_cleaned.dropna(subset='min_distance_cyl_2_1')
df_cleaned = df_cleaned[df_cleaned['min_distance_cyl_2_1'] <= 0]
df_cleaned = df_cleaned.dropna(subset='min_distance_cyl_3_1')
df_cleaned = df_cleaned[df_cleaned['min_distance_cyl_3_1'] <= 0]
df_cleaned = df_cleaned[df_cleaned['a1'] != 360.0]
df_cleaned = df_cleaned[df_cleaned['a2'] != 360.0]
df_cleaned = df_cleaned[df_cleaned['a3'] != 360.0]
#df_cleaned.reset_index(drop=True, inplace=True)
df_cleaned = df_cleaned.round()
print(df_cleaned.shape)
print(df_cleaned.to_string())
#df = df_cleaned

In [ ]:
df_sorted_sym = df_cleaned.sort_values(['min_distance_cyl_1_2', 'min_distance_cyl_1_3','max_distance_cyl_1_2', 'max_distance_cyl_1_3'])
print(df_sorted_sym.to_string()) 

In [ ]:
# Dont use this, too slow

def check_matching_conditions(row1, row2):
    return (
        (row1['min_distance_cyl_1_2'] == row2['min_distance_cyl_1_3']) and
        (row1['max_distance_cyl_1_2'] == row2['max_distance_cyl_1_3']) and
        (row1['min_distance_cyl_3_1'] == row2['min_distance_cyl_2_1']) and
        (row1['max_distance_cyl_3_1'] == row2['max_distance_cyl_2_1']) and
        (row2['min_distance_cyl_1_2'] == row1['min_distance_cyl_1_3']) and
        (row2['max_distance_cyl_1_2'] == row1['max_distance_cyl_1_3']) and
        (row2['min_distance_cyl_3_1'] == row1['min_distance_cyl_2_1']) and
        (row2['max_distance_cyl_3_1'] == row1['max_distance_cyl_2_1'])
    )

# Build groups of matching rows
matched_groups = []  # List to store sets of grouped indices
visited_indices = set()  # Track indices that have been grouped

for idx1 in df_cleaned.index:
    if idx1 in visited_indices:
        continue  # Skip rows already grouped

    # Start a new group with the current row
    group = {idx1}

    # Check all other rows for matches
    for idx2 in df_cleaned.index:
        if idx2 != idx1 and idx2 not in visited_indices:
            # Check if this row matches any row already in the group
            if any(check_matching_conditions(df_cleaned.loc[other], df_cleaned.loc[idx2]) for other in group):
                group.add(idx2)

    # Add the group to the list of matched groups if it's larger than 1
    if len(group) > 1:
        matched_groups.append(group)
        visited_indices.update(group)

# Build the resulting DataFrame with groups in order
sorted_df = pd.DataFrame()
for group in matched_groups:
    group_df = df_cleaned.loc[list(group)]  # Extract rows for this group
    sorted_df = pd.concat([sorted_df, group_df], ignore_index=True)

# Display results
if not sorted_df.empty:
    print(f"Number of matched groups: {len(matched_groups)}")
    print(sorted_df.to_string())
else:
    print("No matching groups found.")

In [ ]:
# Dont use this
def check_matching_conditions(row1, row2):
    return (
        (row1['min_distance_cyl_1_2'] == row2['min_distance_cyl_1_3']) and
        (row1['max_distance_cyl_1_2'] == row2['max_distance_cyl_1_3']) and
        (row1['min_distance_cyl_3_1'] == row2['min_distance_cyl_2_1']) and
        (row1['max_distance_cyl_3_1'] == row2['max_distance_cyl_2_1']) and
        (row2['min_distance_cyl_1_2'] == row1['min_distance_cyl_1_3']) and
        (row2['max_distance_cyl_1_2'] == row1['max_distance_cyl_1_3']) and
        (row2['min_distance_cyl_3_1'] == row1['min_distance_cyl_2_1']) and
        (row2['max_distance_cyl_3_1'] == row1['max_distance_cyl_2_1'])
    )

# Build groups of matching rows
matched_groups = []  # List to store sets of grouped indices
visited_indices = set()  # Track indices that have been grouped

for idx1 in df_cleaned.index:
    if idx1 in visited_indices:
        continue  # Skip rows already grouped

    # Start a new group with the current row
    group = {idx1}

    # Check all other rows for matches
    for idx2 in df_cleaned.index:
        if idx2 != idx1 and idx2 not in visited_indices:
            # Check if this row matches any row already in the group
            if any(check_matching_conditions(df_cleaned.loc[other], df_cleaned.loc[idx2]) for other in group):
                group.add(idx2)

    # Add the group to the list of matched groups if it's larger than 1
    if len(group) > 1:
        matched_groups.append(group)
        visited_indices.update(group)

# Add group identifiers to the DataFrame
grouped_rows = []  # List to hold rows with group assignment
for group_id, group in enumerate(matched_groups, start=1):
    group_df = df_cleaned.loc[list(group)].copy()  # Extract rows for this group
    group_df.insert(3, 'group', group_id)  # Insert the 'group' column after the 'a3' column
    grouped_rows.append(group_df)

# Combine all grouped DataFrames
sorted_df = pd.concat(grouped_rows, ignore_index=True)

# Display results
if not sorted_df.empty:
    print(f"Number of matched groups: {len(matched_groups)}")
    print(sorted_df.to_string())
else:
    print("No matching groups found.")

In [ ]:
def check_matching_conditions_numpy(row1, row2):
    """Compares two rows (numpy arrays) based on specific conditions."""
    return (
        (row1[0] == row2[4]) and
        (row1[1] == row2[5]) and
        (row1[6] == row2[2]) and
        (row1[7] == row2[3]) and
        (row2[0] == row1[4]) and
        (row2[1] == row1[5]) and
        (row2[6] == row1[2]) and
        (row2[7] == row1[3])
    )

def build_groups_optimized(df):
    """Efficiently builds groups of matching rows."""
    matched_groups = []
    visited_indices = set()

    # Convert the relevant columns to a numpy array for faster access
    data = df[
        [
            'min_distance_cyl_1_2', 'max_distance_cyl_1_2',
            'min_distance_cyl_2_1', 'max_distance_cyl_2_1',
            'min_distance_cyl_1_3', 'max_distance_cyl_1_3',
            'min_distance_cyl_3_1', 'max_distance_cyl_3_1',
        ]
    ].to_numpy()

    for idx1 in range(len(data)):
        if idx1 in visited_indices:
            continue  # Skip rows already grouped

        group = {idx1}
        visited_indices.add(idx1)

        for idx2 in range(len(data)):
            if idx2 != idx1 and idx2 not in visited_indices:
                if check_matching_conditions_numpy(data[idx1], data[idx2]):
                    group.add(idx2)
                    visited_indices.add(idx2)

        if len(group) > 1:
            matched_groups.append(group)

    return matched_groups

def add_group_column_optimized(df, matched_groups):
    """Add group identifiers to the DataFrame."""
    # Create a group column with default value 0 (ungrouped)
    group_column = [0] * len(df)

    # Assign group IDs
    for group_id, group in enumerate(matched_groups, start=1):
        for idx in group:
            group_column[idx] = group_id

    # Add the group column to the DataFrame
    df['group'] = group_column

    # Move the 'group' column to the third position
    group_col = df.pop('group')  # Remove the 'group' column
    df.insert(3, 'group', group_col)  # Insert it at the third position
    return df
# Assuming `df_cleaned` is your DataFrame
matched_groups = build_groups_optimized(df_cleaned)
sorted_df = add_group_column_optimized(df_cleaned, matched_groups)

# Display results
if not sorted_df.empty:
    print(f"Number of matched groups: {len(matched_groups)}")
    print(sorted_df.to_string())
else:
    print("No matching groups found.")

In [ ]:
sorted_df = sorted_df.sort_values(by='group')
print(sorted_df.to_string())

In [ ]:
df_cleaned = df_cleaned.sort_values(by=['a1', 'a2', 'a3'], ascending=[False, True, True])
# Step 2: Remove all but the first row of each group, keeping rows where 'a1' == 0

rows_to_keep = set()  # This will store indices of rows to keep

# Step 1: Add rows with group == 0 to rows_to_keep
group_0_indices = df_cleaned[df_cleaned['group'] == 0].index
rows_to_keep.update(group_0_indices)

# Step 2: Process matched_groups and add the first row of each group
for group_id, group in enumerate(matched_groups, start=1):
    group_df = df_cleaned.loc[list(group)].sort_values(by=['a1', 'a2'], ascending=[True, False])
    
    # Keep only the first row of the sorted group
    rows_to_keep.add(group_df.iloc[0].name)

# Step 3: Identify rows not in any group
all_indices = set(df_cleaned.index)
rows_in_groups = rows_to_keep
rows_not_in_groups = all_indices - rows_in_groups

# Step 4: Create the final DataFrame by keeping only the rows not in groups
df_final = df_cleaned.loc[list(rows_to_keep)].copy()

# Optionally, add the rows from the sorted groups back if needed
# df_final = pd.concat([df_final, sorted_df], ignore_index=True).sort_values(by=['a1', 'a2', 'a3'])

# Display the result
print(df_final.shape)
print(df_final.to_string())

In [ ]:

df_cleaned = sorted_df.copy()
# Step 1: Add rows with group == 0 to rows_to_keep
group_0_indices = df_cleaned[df_cleaned['group'] == 0].index
rows_to_keep.update(group_0_indices)

# Step 2: Process matched_groups and add the first row of each group
for group_id, group in enumerate(matched_groups, start=1):
    group_df = df_cleaned.loc[list(group)].sort_values(by=['a1', 'a2'], ascending=[True, False])
    
    # Keep only the first row of the sorted group
    rows_to_keep.add(group_df.iloc[0].name)

# Step 2: Identify rows not in any group
all_indices = set(df_cleaned.index)
rows_in_groups = rows_to_keep
rows_not_in_groups = all_indices - rows_in_groups

# Step 3: Create the final DataFrame by keeping only the rows not in groups
df_final = df_cleaned.loc[list(rows_to_keep)].copy()

# Optionally, add the rows from the sorted groups back if needed
# df_final = pd.concat([df_final, sorted_df], ignore_index=True).sort_values(by=['a1', 'a2', 'a3'])

# Step 4: Prepare the data for the scatter plots
# Create separate DataFrames for the three sets of points
df_group_0 = df_cleaned.loc[df_cleaned['group'] == 0]
df_other_groups = df_cleaned.loc[list(rows_to_keep)]
df_dropped = df_cleaned.loc[list(rows_not_in_groups)]

# Step 4: Create the scatter plots separately
# Create the first scatter plot for group 0
plt.figure(figsize=(12, 8))
plt.scatter(df_group_0['a1'], df_group_0['a2'], label='Group 0 (a2)', color='b', marker='o')
plt.scatter(df_group_0['a1'], df_group_0['a3'], label='Group 0 (a3)', color='r', marker='x')
plt.title('Group 0')
plt.xlabel('a1')
plt.ylabel('a2, a3')
plt.grid(True)
plt.xticks(np.arange(0, 361, 30))  # X-axis (A1) from 0 to 360, steps of 30
plt.yticks(np.arange(-60, 421, 30))  # Y-axis (A2, A3) from -60 to 420, steps of 30
plt.legend()
plt.show()

# Create the second scatter plot for other groups
plt.figure(figsize=(12, 8))
plt.scatter(df_other_groups['a1'], df_other_groups['a2'], label='Other Groups (a2)', color='g', marker='o')
plt.scatter(df_other_groups['a1'], df_other_groups['a3'], label='Other Groups (a3)', color='purple', marker='x')
plt.title('Other Groups')
plt.xlabel('a1')
plt.ylabel('a2, a3')
plt.grid(True)
plt.xticks(np.arange(0, 361, 30))  # X-axis (A1) from 0 to 360, steps of 30
plt.yticks(np.arange(-60, 421, 30))  # Y-axis (A2, A3) from -60 to 420, steps of 30
plt.legend()
plt.show()

# Create the third scatter plot for dropped rows
plt.figure(figsize=(12, 8))
plt.scatter(df_dropped['a1'], df_dropped['a2'], label='Dropped (a2)', color='orange', marker='o')
plt.scatter(df_dropped['a1'], df_dropped['a3'], label='Dropped (a3)', color='pink', marker='x')
plt.title('Dropped Rows')
plt.xlabel('a1')
plt.ylabel('a2, a3')
plt.grid(True)
plt.xticks(np.arange(0, 361, 30))  # X-axis (A1) from 0 to 360, steps of 30
plt.yticks(np.arange(-60, 421, 30))  # Y-axis (A2, A3) from -60 to 420, steps of 30
plt.legend()
plt.show()

print(df_final.sort_values(by='group').to_string())

In [ ]:
# Define the function to calculate the wrapped range for a given column
def calculate_wrapped_range(group):
    # For a2 column
    sorted_a2 = np.sort(group['a2'].values)
    extended_a2 = np.concatenate((sorted_a2, sorted_a2 + 360))
    gaps_a2 = np.diff(extended_a2)
    max_gap_idx_a2 = np.argmax(gaps_a2)
    start_angle_a2 = extended_a2[max_gap_idx_a2 + 1] % 360
    end_angle_a2 = extended_a2[max_gap_idx_a2] % 360
    
    # For a3 column
    sorted_a3 = np.sort(group['a3'].values)
    extended_a3 = np.concatenate((sorted_a3, sorted_a3 + 360))
    gaps_a3 = np.diff(extended_a3)
    max_gap_idx_a3 = np.argmax(gaps_a3)
    start_angle_a3 = extended_a3[max_gap_idx_a3 + 1] % 360
    end_angle_a3 = extended_a3[max_gap_idx_a3] % 360
    
    # Return a2_min, a2_max, a3_min, and a3_max
    return pd.Series({'a2_min': start_angle_a2, 'a2_max': end_angle_a2, 'a3_min': start_angle_a3, 'a3_max': end_angle_a3})

# Group by 'a1' and apply the function
df_cleaned_copy = df_final.copy()
angular_ranges_df = df_cleaned_copy.groupby('a1').apply(calculate_wrapped_range).reset_index()

# Display the result
df_string = angular_ranges_df.to_string()
print(df_string)

In [ ]:
# Save the DataFrame as a CSV file with semicolon delimiters
#csv_path = "./angular_ranges_sym.csv"  # File will be saved in the same folder as the notebook
#angular_ranges_df.round().to_csv(csv_path, sep=';', index=False, header=False)

In [ ]:
# Define the function to plot the ranges
def plot_ranges(df):
    """
    This function takes a dataframe with columns ['a1', 'a2_min', 'a2_max', 'a3_min', 'a3_max']
    and plots the blower ranges for both sides (a2 and a3) based on a1 values.
    """
    # Adjust a2_max if it's smaller than a2_min (wrap-around for a2)
    wrapped_df = df.copy()
    
    wrapped_df['a2_max'] = wrapped_df.apply(
        lambda row: row['a2_max'] + 360 if row['a2_max'] < row['a2_min'] else row['a2_max'],
        axis=1
    )
    
    # Identify large gaps and create a grouping based on a1
    step_size = wrapped_df['a1'].diff().dropna().min()
    wrapped_df['gap'] = wrapped_df['a1'].diff().fillna(step_size) > step_size
    wrapped_df['group'] = wrapped_df['gap'].cumsum()

    # Plot the a2 ranges
    plt.figure(figsize=(12, 8))

    for group, group_data in wrapped_df.groupby('group'):
        plt.fill_between(
            group_data['a1'].to_numpy(),
            group_data['a2_min'].to_numpy(),
            group_data['a2_max'].to_numpy(),
            alpha=0.5,
            color='skyblue')
        plt.plot(group_data['a1'].to_numpy(), group_data['a2_min'].to_numpy(), color='blue')
        plt.plot(group_data['a1'].to_numpy(), group_data['a2_max'].to_numpy(), color='red')

    # Now adjust a3_min if it's smaller than a3_max (wrap-around for a3)
    wrapped_df['a3_min'] = wrapped_df.apply(
        lambda row: row['a3_min'] - 360 if row['a3_max'] < row['a3_min'] else row['a3_min'],
        axis=1
    )

    # Create a new group for a3 ranges
    wrapped_df['gap'] = wrapped_df['a1'].diff().fillna(step_size) > step_size
    wrapped_df['group'] = wrapped_df['gap'].cumsum()

    # Plot the a3 ranges
    for group, group_data in wrapped_df.groupby('group'):
        plt.fill_between(
            group_data['a1'].to_numpy(),
            group_data['a3_min'].to_numpy(),
            group_data['a3_max'].to_numpy(),
            alpha=0.5,
            color='lightgreen')
        plt.plot(group_data['a1'].to_numpy(), group_data['a3_min'].to_numpy(), 'g--')
        plt.plot(group_data['a1'].to_numpy(), group_data['a3_max'].to_numpy(), 'm--')

    # Labels and Titles
    plt.xlabel("Measure (degrees)")
    plt.ylabel("Blower Range (degrees)")
    plt.title("Blower Ranges for Different A1 Values (Both Sides)")
    plt.legend()
    plt.grid(True)
    plt.xticks(np.arange(0, 361, 30))  # X-axis (A1) from 0 to 360, steps of 30
    plt.yticks(np.arange(-60, 421, 30))  # Y-axis (A2) from 0 to 420, steps of 30

    # Show the plot
    plt.show()

# Example of how to use the function:
# Assuming df_cleaned has the previously defined columns ['a1', 'a2_min', 'a2_max', 'a3_min', 'a3_max']
plot_ranges(angular_ranges_df)

In [ ]:
combined = pd.concat([df.round(), df_cleaned], ignore_index=True)
unique_rows = combined.drop_duplicates(keep=False)
print(unique_rows.shape)
print(unique_rows.to_string())

Verify symmetries by finding a mappiinf/rotation such that it recreates the full search space
make a google sheets for combinations: ns^3*na^2*nm+ns^2*na*nm  if 3 angles are measured per measured angle, only times 3, not also times measured angle